<a href="https://colab.research.google.com/github/mostume222/inf-test/blob/master/infa_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Step 0.** If for some reason you encounter issues navigating the folders in the project or having errors you can delete the project folder in the online colab environment by using the following code cell. If you are just starting to use this fresh colab notebook skip this step.

In [ ]:
!cd ~
!pwd
!ls
!rm -r inf-test/

/content
sample_data
rm: cannot remove 'inf-test/': No such file or directory


**Step 1.** Clone github repository from link https://github.com/mostume222/inf-test.git , this will download the source code and also make the correct project structure to tun the compression program.

In [1]:
!git clone https://github.com/mostume222/inf-test.git
%cd inf-test
!git checkout master #go to master branch to have the correct project structure
!ls

Cloning into 'inf-test'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 164 (delta 64), reused 111 (delta 31), pack-reused 0 (from 0)
Receiving objects: 100% (164/164), 37.69 MiB | 6.18 MiB/s, done.
Resolving deltas: 100% (64/64), done.
/content/inf-test
Branch 'master' set up to track remote branch 'master' from 'origin'.
Switched to a new branch 'master'
data  env  infa_main.ipynb  output_compression	README.md  sequences  src


**Step 2.** Install the conda environment in /inf-test/env/infa.yml to install dependencies neded to run the compression procedure.

If you would like to install your own dependencies, the important ones are:

* cd-hit 4.8.1  
* bowtie2 2.5.4
* cutadapt 3.5
* python 3.6.13
* perl 5.34.0
* pip 21.2.2

In [2]:
#install condacolab
!ls
!pip install -q condacolab
import condacolab
condacolab.install()
!ls
#create conda envionment in infa.yml file
!conda env create -f env/infa.yml --quiet
!conda env list

data  env  infa_main.ipynb  output_compression	README.md  sequences  src
⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:12
🔁 Restarting kernel...
condacolab_install.log	data  env  infa_main.ipynb  output_compression	README.md  sequences  src
Channels:
 - defaults
 - conda-forge
 - bioconda
Platform: linux-64
Solving environment: ...working... done
Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...working... done
# conda environments:
#
base                     /usr/local
infc                     /usr/local/envs/infc



**Step 3.** Run the following cell to run the compression procedure to generate the pangenome sequences of the input sequence.

In [ ]:
# upload your own file
from google.colab import files
import os
import shutil

# Create the directory if it doesn't exist
destination_dir = '/content/inf-test/sequences/'
os.makedirs(destination_dir, exist_ok=True)

# Upload the file
uploaded = files.upload()

# Move the uploaded file to the destination directory
for filename in uploaded.keys():
    shutil.move(filename, os.path.join(destination_dir, filename))

print(f"File(s) successfully moved to {destination_dir}")


In [1]:
#  run the script with conda env
%cd /content/inf-test/src/compressor/
!source activate infc && ls
!chmod +x run_compress.sh
!source activate infc && ./run_compress.sh 0.94 0.958 200 50 100000 1,.comp


/content/inf-test/src/compressor
cluster_acc_family.pl	    divider.pl		       lucas_compressor.sh
cluster_acc.pl		    enumerate.pl	       modify_headers.pl
cluster_acc_variant.pl	    enumerate.py	       remove.sh
cluster_element_counter.pl  enumerate.sh	       run_compress.sh
concat_clusters.sh	    kmer.pl		       uniques_concatenation.pl
concat_clusters_test.sh     label_clusters_variant.sh  uniques_entanglement.pl
counter.pl		    lucas_back		       uniques_gitrog.pl
count_lenghts_pieces.pl     lucas_compressor_b.sh      uniques_intanglement.pl
1,.comp
This is cutadapt 3.5 with Python 3.6.13
Command line parameters: --max-n 0 -o /content/inf-test/src/compressor/../..//output_compression/1,.comp//kmers_clean.fasta /content/inf-test/src/compressor/../..//output_compression/1,.comp//kmers_w.fasta
Processing reads on 1 core in single-end mode ...
Done           00:00:02       405,046 reads @   6.4 µs/read;   9.41 M reads/minute
Finished in 2.58 s (6 us/read; 9.41 M reads/minute).

=== Sum

In [ ]:
#clean
!rm -r /content/inf-test/output_compression/1,.comp/

rm: cannot remove '/content/inf-test/output_compression/1,.comp/': No such file or directory
